In [1]:
class Model:
    def __init__(self,sess,name):
        self.sess=sess
        self.name=name
        self._build_net()
    def _build_net(self):#변수 설정 및 network 구성
        with tf.variable_scope(self.name):
            self.learning_rate=0.001
            self.keep_prob=tf.placeholder(tf.float32)
            self.X=tf.placeholder(tf.float32,shape=[None,784])
            self.X_img = tf.reshape(self.X, [-1, 28, 28, 1])   # img 28x28x1 (black/white)
            self.Y = tf.placeholder(tf.float32, [None, 10])
            self.W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
            self.W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
            self.W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
            self.FW1= tf.get_variable("FW1", shape=[128 * 4 * 4, 625],
                     initializer=tf.contrib.layers.xavier_initializer())
            self.FW2= tf.get_variable("FW2", shape=[625, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
            self.Fb1 = tf.Variable(tf.random_normal([625]))
            self.Fb2 = tf.Variable(tf.random_normal([10]))
        #convolution layer
        self.L1 = tf.nn.conv2d(self.X_img, self.W1, strides=[1, 1, 1, 1], padding='SAME')
        self.L1 = tf.nn.relu(self.L1)
        self.L1 = tf.nn.max_pool(self.L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
        self.L1 = tf.nn.dropout(self.L1,keep_prob=self.keep_prob)
            
        self.L2 = tf.nn.conv2d(self.L1, self.W2, strides=[1, 1, 1, 1], padding='SAME')
        self.L2 = tf.nn.relu(self.L2)
        self.L2 = tf.nn.max_pool(self.L2, ksize=[1, 2, 2, 1],
                strides=[1, 2, 2, 1], padding='SAME')
        self.L2 = tf.nn.dropout(self.L2,keep_prob=self.keep_prob)
            
        self.L3 = tf.nn.conv2d(self.L2, self.W3, strides=[1, 1, 1, 1], padding='SAME')
        self.L3 = tf.nn.relu(self.L3)
        self.L3 = tf.nn.max_pool(self.L3, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
        self.L3 = tf.nn.dropout(self.L3,keep_prob=self.keep_prob)
        self.L3_flat = tf.reshape(self.L3, [-1, 128 * 4 * 4])
            
            
        #fully connected layer
        self.FL1 = tf.nn.relu(tf.matmul(self.L3_flat, self.FW1) + self.Fb1)
        self.FL1 = tf.nn.dropout(self.FL1, keep_prob=self.keep_prob)
            
        self.FL2 = tf.matmul(self.FL1, self.FW2)+self.Fb2
            
        #cost & optimizer
        self.cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.FL2, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
            
        #accuracy & prediction
        self.correct_prediction = tf.equal(tf.argmax(self.FL2, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
    def predicted(self,x_test,keep_prob=1.0):
        return self.sess.run(self.FL2,feed_dict={self.X:x_test,self.keep_prob:keep_prob})
    def get_accuracy(self,x_test,y_test,keep_prob=1.0):
        return self.sess.run(self.accuracy,feed_dict={self.X:x_test,self.Y:y_test,self.keep_prob:keep_prob})
    def train(self,x_data,y_data,keep_prob=0.7):
        return self.sess.run([self.cost,self.optimizer],feed_dict={self.X:x_data,self.Y:y_data,self.keep_prob:keep_prob})
            

In [2]:
import tensorflow as tf
import random
# import matplotlib.pyplot as plt
import time
from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility
#load mnist data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess=tf.Session()
m1=Model(sess,"m1")

training_epochs = 15
batch_size = 100

sess.run(tf.global_variables_initializer())
start_time=time.time()
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
       
        c , _ = m1.train(batch_xs,batch_ys,0.7)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')
print('learning time : ',time.time()-start_time)




C:\Users\Robotmedia9\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.359114391
Epoch: 0002 cost = 0.098635341
Epoch: 0003 cost = 0.073617802
Epoch: 0004 cost = 0.059579738
Epoch: 0005 cost = 0.050943636
Epoch: 0006 cost = 0.045958366
Epoch: 0007 cost = 0.042547446
Epoch: 0008 cost = 0.038630164
Epoch: 0009 cost = 0.036922806
Epoch: 0010 cost = 0.033910994
Epoch: 0011 cost = 0.031285284
Epoch: 0012 cost = 0.031118160
Epoch: 0013 cost = 0.029283639
Epoch: 0014 cost = 0.026602107
Epoch: 0015 cost = 0.026624620
Learning Finished!
learning time :  76.50883293151855


In [3]:
print('accuracy : ',m1.get_accuracy(mnist.test.images,mnist.test.labels,1))
for i in range(10):
    # Get one and predict
    r = random.randint(0, mnist.test.num_examples - 1)
    print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
    print("Prediction: ",sess.run(tf.argmax(m1.predicted(mnist.test.images[r:r + 1],1),1)))


accuracy :  0.9938
Label:  [8]
Prediction:  [8]
Label:  [0]
Prediction:  [0]
Label:  [2]
Prediction:  [2]
Label:  [0]
Prediction:  [0]
Label:  [2]
Prediction:  [2]
Label:  [0]
Prediction:  [0]
Label:  [1]
Prediction:  [1]
Label:  [7]
Prediction:  [7]
Label:  [6]
Prediction:  [6]
Label:  [6]
Prediction:  [5]
